# Scrapy를 사용한 크롤링

웹 크롤러를 만드는 과정에서 반복 작업을 줄여주는 라이브러리

## Scrapy

스크래파이는

- 웹사이트의 링크를 찾아서 분석하고,
- 도메인이나 도메인 목록 크롤링 작업을 쉽게 해주는 라이브러리

### 다운로드

- [공식 웹사이트](http://scrapy.org/download/)에서 직접 다운
- 인스톨 관리자(pip 등)를 사용해서 다운
    - (나의 경우) __conda에서 다운__

### 프로젝트 생성

In [8]:
! scrapy startproject wikiSpider

New Scrapy project 'wikiSpider', using template directory 'C:\Users\User\anaconda3\envs\web_venv\lib\site-packages\scrapy\templates\project', created in:
    C:\Users\User\Desktop\web_scrapping\0. Notebooks\wikiSpider

You can start your first spider with:
    cd wikiSpider
    scrapy genspider example example.com


명령어를 실행하게 되면 'wikiSpider'라는 새 프로젝트가 생성된다.

이 디렉터리는 다음과 같은 구조로 되어있다.

In [23]:
! tree /F wikiSpider

폴더 PATH의 목록입니다.
볼륨 일련 번호는 3ADA-D4B8입니다.
C:\USERS\USER\DESKTOP\WEB_SCRAPPING\0. NOTEBOOKS\WIKISPIDER
│  scrapy.cfg
│  
└─wikiSpider
    │  items.py
    │  middlewares.py
    │  pipelines.py
    │  settings.py
    │  __init__.py
    │  
    ├─spiders
    │  │  __init__.py
    │  │  
    │  └─__pycache__
    └─__pycache__


- /spiders
    - 어떤 웹사이트를 어떻게 크롤링할 것인가를 명시하고, 각각의 웹 페이지의 어떤 부분을 스크래핑할 것인지 명시하는 클래스가 모여있는 디렉토리
    - 여러개의 spider.py 파일을 만들어 사용할 수 있음

- items.py
    - 크롤링하는 데이터에 대해 정의하는 클래스가 있는 파일(MVC → M)

- pipelines.py
    - item 객체 형태로 크롤링을 하고 출력하기 전에 item을 받아서 실행하는 파일이 정의되어 있는 파일
    - item을 자유롭게 가공하거나 다양한 파일 형태로 저장할 수 있도록 하는 클래스

- settings.py
    - spider나 item pipeline 등이 어떻게 동작하게 할지에 대한 세부적 설정이 담겨 있는 파일
        e.g. robots.txt 정책을 따를 것인지 안 따를 것인지, pipeline을 사용할지 안 할지


### 크롤러 만들기

- wikiSpider/ 에 있는 **items.py** 파일에 새 항목 Ariticle을 정의
- wikiSpider/spiders/ 안에 새 파일 **articleSpider.py**를 추가

In [27]:
%%writefile wikiSpider/wikiSpider/items.py

# -*- coding: utf-8 -*-

# Define here the models for your scraped items
#
# See documentation in:
# https://doc.scrapy.org/en/latest/topics/items.html

from scrapy import Item, Field


class Article(Item):
    # define the fields for your item here like:
    # name = scrapy.Field()
    title = Field()

Overwriting wikiSpider/wikiSpider/items.py


Item 객체 하나가 웹사이트의 페이지 하나에 대응

단순히 각 페이지에서 title 필드만 수집

In [46]:
%%writefile  wikiSpider/wikiSpider/spiders/articleSpider.py

from scrapy.selector import Selector
from scrapy import Spider
from wikiSpider.items import Article


class ArticleSpider(Spider):
    name="article"
    allowed_domains = ["en.wikipedia.org"]
    start_urls = ["http://en.wikipedia.org/wiki/Main_Page",
                  "https://en.wikipedia.org/wiki/Python_%28programming_language%29"]

    def parse(self, response):
        item = Article()
        title = response.xpath('//h1/text()')[0].extract()
        print("Title is: " + title)
        item['title'] = title
        return item

Overwriting wikiSpider/wikiSpider/spiders/articleSpider.py


잘 추가 되었는가 확인

In [30]:
! tree /F wikiSpider

폴더 PATH의 목록입니다.
볼륨 일련 번호는 3ADA-D4B8입니다.
C:\USERS\USER\DESKTOP\WEB_SCRAPPING\0. NOTEBOOKS\WIKISPIDER
│  scrapy.cfg
│  
├─.ipynb_checkpoints
└─wikiSpider
    │  items.py
    │  middlewares.py
    │  pipelines.py
    │  settings.py
    │  __init__.py
    │  
    ├─spiders
    │  │  articleSpider.py
    │  │  __init__.py
    │  │  
    │  └─__pycache__
    └─__pycache__


ArticleSpider 안의 name="article" 로 스크레이퍼를 호출

그전에 현재 경로를 wikiSpider 폴더로 옮겨가자.

In [47]:
%cd wikiSpider

C:\Users\User\Desktop\web_scrapping\0. Notebooks\wikiSpider


In [48]:
! scrapy crawl article

Title is: Python (programming language)
Title is: Main Page


2020-04-24 00:11:42 [scrapy.utils.log] INFO: Scrapy 1.6.0 started (bot: wikiSpider)
2020-04-24 00:11:42 [scrapy.utils.log] INFO: Versions: lxml 4.5.0.0, libxml2 2.9.9, cssselect 1.1.0, parsel 1.5.2, w3lib 1.21.0, Twisted 20.3.0, Python 3.8.2 (default, Apr 14 2020, 19:01:40) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 19.1.0 (OpenSSL 1.1.1g  21 Apr 2020), cryptography 2.8, Platform Windows-10-10.0.18362-SP0
2020-04-24 00:11:42 [scrapy.crawler] INFO: Overridden settings: {'BOT_NAME': 'wikiSpider', 'NEWSPIDER_MODULE': 'wikiSpider.spiders', 'ROBOTSTXT_OBEY': True, 'SPIDER_MODULES': ['wikiSpider.spiders']}
2020-04-24 00:11:42 [scrapy.extensions.telnet] INFO: Telnet Password: 36ac62775d8af95d
2020-04-24 00:11:42 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']
2020-04-24 00:11:42 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.robotstxt

csv 파일로 저장해 보자.

In [49]:
! scrapy crawl article -o articles.csv -t csv

Title is: Python (programming language)
Title is: Main Page


2020-04-24 00:13:50 [scrapy.utils.log] INFO: Scrapy 1.6.0 started (bot: wikiSpider)
2020-04-24 00:13:50 [scrapy.utils.log] INFO: Versions: lxml 4.5.0.0, libxml2 2.9.9, cssselect 1.1.0, parsel 1.5.2, w3lib 1.21.0, Twisted 20.3.0, Python 3.8.2 (default, Apr 14 2020, 19:01:40) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 19.1.0 (OpenSSL 1.1.1g  21 Apr 2020), cryptography 2.8, Platform Windows-10-10.0.18362-SP0
2020-04-24 00:13:50 [scrapy.crawler] INFO: Overridden settings: {'BOT_NAME': 'wikiSpider', 'FEED_FORMAT': 'csv', 'FEED_URI': 'articles.csv', 'NEWSPIDER_MODULE': 'wikiSpider.spiders', 'ROBOTSTXT_OBEY': True, 'SPIDER_MODULES': ['wikiSpider.spiders']}
2020-04-24 00:13:50 [scrapy.extensions.telnet] INFO: Telnet Password: f67bf472607c557c
2020-04-24 00:13:50 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2020-04-24 00:13:50 [s